In [3]:
from numba import cfunc
from numba.types import Record, CPointer, void, double, uint64
from pygraphblas import Matrix, BinaryOp, lib, ffi as gbffi
from cffi import FFI


In [10]:

src = """
typedef struct BF_tuple {
    double    w;
    uint64_t  h;
    uint64_t pi;
} BF_tuple;

typedef void (*bf_min)(BF_tuple*, BF_tuple*, BF_tuple*);
typedef void (*bf_plus)(BF_tuple*, BF_tuple*, BF_tuple*);

#define INFINITY ...
"""
from numba import cfunc, carray
from numba import cffi_support

ffi = FFI()
ffi.cdef(src)
ffi.set_source('_bf', """
#include <math.h>
#include <stdint.h>
""")

def new_tuple(w, h, pi):
    data = ffi.new('BF_tuple[1]')
    ptr = ffi.cast('BF_tuple*', data)
    ptr[0].w = w
    ptr[0].h = h
    ptr[0].pi = pi
    addr = int(ffi.cast('size_t', ptr))
    return addr
    
cffi_support.map_type(ffi.typeof('BF_tuple'), use_record_dtype=True)
min_sig = cffi_support.map_type(ffi.typeof('bf_min'), use_record_dtype=True)
plus_sig = cffi_support.map_type(ffi.typeof('bf_plus'), use_record_dtype=True)

@cfunc(min_sig)
def bf_min(z_, x_, y_):
    z = carray(z_, 1)[0]
    x = carray(x_, 1)[0]
    y = carray(y_, 1)[0]
    z.w = x.w if x.w < y.w else y.w

@cfunc(plus_sig)
def bf_plus(z_, x_, y_):
    z = carray(z_, 1)[0]
    x = carray(x_, 1)[0]
    y = carray(y_, 1)[0]
    z.w = x.w - y.w
    
def new_udt(name):
    t = gbffi.new('GrB_Type*')
    lib.GrB_Type_new(t, ffi.sizeof(name))
    return t

bf = new_udt('BF_tuple')

def new_binop(func, typ):
    o = gbffi.new('GrB_BinaryOp*')
    typ = typ[0]
    lib.GrB_BinaryOp_new(o, gbffi.cast('GxB_binary_function', func.address), typ, typ, typ)
    return o

op = new_binop(bf_plus, bf)

op = BinaryOp('plus', 'bf', op[0])

A = Matrix.from_type(bf[0], 10, 10)
B = Matrix.from_type(bf[0], 10, 10)

z = new_tuple(0.1, 1, 1)
x = new_tuple(0.2, 2, 2)
y = new_tuple(0.3, 3, 3)

A[0,0] = z
B[1,1] = x

with op:
    print((A + B).nvals)

KeyError: <cdata 'struct GB_Type_opaque *' 0x5639d26c4d00>

In [9]:
ffi.INFINITY

AttributeError: 'FFI' object has no attribute 'INFINITY'

In [58]:
double.reflected

False

In [60]:
"""
Demonstrate potential numba feature for casting an arbitrary pointer into
a jitclass instance.
"""

from collections import OrderedDict

import numpy as np

from numba import njit, jitclass, types
from numba import extending
from numba import cgutils

#
# Implement the core logic of the casting function.
#


def _unsafe_cast_ptr_to_class(int_type, class_type):
    inst_typ = class_type.instance_type
    sig = inst_typ(types.voidptr, class_type)

    def codegen(context, builder, signature, args):
        ptr, _ = args
        alloc_type = context.get_data_type(inst_typ.get_data_type())

        inst_struct = context.make_helper(builder, inst_typ)
        # Set meminfo to NULL
        inst_struct.meminfo = cgutils.get_null_value(inst_struct.meminfo.type)
        # Set data from the given pointer
        inst_struct.data = builder.bitcast(ptr, alloc_type.as_pointer())

        return inst_struct._getvalue()

    return sig, codegen

#
# Make an intrinsic for our unsafe_cast function
#

@extending.intrinsic
def unsafe_cast(typingctx, src, dst):
    # This is the typing logic
    if isinstance(src, types.Integer) and isinstance(dst, types.ClassType):
        # This defines the codegen logic
        return _unsafe_cast_ptr_to_class(src, dst)
    raise TypeError


#
# Make a class
#

spec = OrderedDict()
spec['intval'] = types.int32
spec['realval'] = types.float32


@jitclass(spec)
class Stuff(object):
    def __init__(self, x, y):
        self.intval = x
        self.realval = y

    def get(self):
        return (self.intval, self.realval)

#
# A function to exercise our intrinsic and cast a pointer to a Stuff instance
#


@njit
def foo(ptr):
    # unsafe_cast is an intrinsic that can only be called inside a jit-ed function
    return unsafe_cast(ptr, Stuff)


def main():
    ary = np.zeros(1, dtype=np.dtype([('x', 'int32'), ('y', 'float32')]))
    ary[0]['x'] = 123
    ary[0]['y'] = 3.14
    ptr = ary.ctypes.data
    obj = foo(ptr)

    print(obj.get())

    print('modify the obj')
    obj.intval *= 100
    print(obj.get(), ary)

    print('modify the array')
    ary[0]['y'] *= 2
    print(obj.get(), ary)
    return obj


o = main()


(123, 3.140000104904175)
modify the obj
(12300, 3.140000104904175) [(12300, 3.14)]
modify the array
(12300, 6.28000020980835) [(12300, 6.28)]


In [69]:
o._numba_type_.jitmethods['get'].py_func

<function __main__.Stuff.get(self)>